In [352]:
import csv
import re
import sys
import time
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn import metrics, tree
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, f1_score, precision_score,
                             recall_score)
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.feature_selection import chi2, SelectKBest

In [353]:
df = pd.read_csv('dataset.tsv', sep='\t', quoting=csv.QUOTE_NONE, dtype=str, encoding = 'utf-8',
                 header=None, names=["instance", "text", "id", "sentiment", "is_sarcastic"])
# df = shuffle(df)

In [354]:
text_data = np.array([])
# Read tweets
for text in df.text:
    text_data = np.append(text_data, text)

In [355]:
""" Functions for text pre-processing """


def remove_URL(sample):
    """Remove URLs from a sample string"""
    return re.sub(r"http\S+", " ", sample)


def remove_punctuation(sample):
    """Remove punctuations from a sample string"""
#     punctuations = r'''$!"&'()*+,-./:;<=>?[\]^`{|}~'''
#     no_punct = ""
#     for char in sample:
#         if char not in punctuations:
#             no_punct = no_punct + char
#     return no_punct
    return re.sub(r'[^\w\s\&\#\@\$\%\_]','',sample)

def myTokenizer(sample):
    """Customized tokenizer"""
    ################################## 1. Remove numbers
    ################################## 2. Remove auspoll thingy
    ################################## 3. Remove starts with au
    new_words = []
    words = sample.split(' ')
    new_words = [word for word in words if len(word) >= 2 and not word.isdigit() and not word.startswith('#aus') and not word.startswith('au')]
    return new_words

def remove_stopwords_NLTK(sample):
    """Remove stopwords using NLTK"""
    stopWords = set(stopwords.words('english'))
    words = [w for w in sample.split(' ') if len(w) >= 2]
    filteredText = ""
    for word in words:
        if word not in stopWords:
            filteredText = filteredText + word + " "
    return filteredText.rstrip()


def porter_stem(sample):
    """Stemming"""
    words = myTokenizer(sample)
    ps = PorterStemmer()
    stemmed_text = ""
    for word in words:
        stemmed_text = stemmed_text + ps.stem(word) + " "
    return stemmed_text.rstrip()


def myPreprocessor(sample):
    """Customized preprocessor"""
    sample = remove_URL(sample)
    sample = sample.lower()
    sample = remove_stopwords_NLTK(sample)
    sample = remove_punctuation(sample)
    sample = porter_stem(sample)
    return sample


In [356]:
#myTokenizer('While the majority of all Internet search engines utilize stop words, they do not prevent a user from using them, but they are ignored.')

In [357]:
# try to use sklearn stop_words later
count = CountVectorizer(preprocessor=myPreprocessor, tokenizer=myTokenizer, max_features=1000)
bag_of_words = count.fit_transform(text_data)
print(count.get_feature_names())
size = len(count.vocabulary_)
print(len(count.vocabulary_))

['#4corner', '#7new', '#abc730', '#abcnews24', '#afpraid', '#agchatoz', '#alp', '#asylumseek', '#betterfutur', '#blackhol', '#bootturnbullout', '#brexit', '#budget2016', '#cfa', '#cfmeu', '#chafta', '#childcar', '#cleanenergi', '#climat', '#climatechang', '#coal', '#corrupt', '#csg', '#csiro', '#csirocut', '#dutton', '#educ', '#election2016', '#environ', '#et', '#faketradi', '#fraudband', '#gonski', '#greatbarrierreef', '#green', '#greens16', '#humanright', '#icac', '#inequ', '#insid', '#labor', '#laborlaunch', '#latelin', '#leadersdeb', '#liber', '#lnp', '#lnpfail', '#malcolm', '#malwar', '#marriageequ', '#medicar', '#nauru', '#nbn', '#nbnco', '#nbngate', '#ndi', '#negativegear', '#npc', '#nswpol', '#panamapap', '#parakeelia', '#peoplesforum', '#pmlive', '#polita', '#putlnplast', '#qanda', '#qldpol', '#refuge', '#renew', '#safeschool', '#savemedicar', '#scomo', '#spendomet', '#springst', '#ssm', '#stopstateterror', '#taxrort', '#tennew', '#thedrum', '#turnbul', '$$$', '$2', '$50b', '$

In [358]:
X = bag_of_words.toarray()
# creating target classes
Y = np.array([])
for text in df.id:
    Y = np.append(Y, text)
# First 1500 for training set, last 500 for test set
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.25, shuffle=False)

In [359]:
clf = MultinomialNB()
# clf = BernoulliNB()
model = clf.fit(X_train, y_train)
text_clf_red = Pipeline([('vect', CountVectorizer(preprocessor=myPreprocessor, tokenizer=myTokenizer)), 
                       ('reducer', SelectKBest(chi2, k=800)),
                       ('clf', MultinomialNB())
                       ])
model_new = text_clf_red.fit(text_data[:1500],Y[:1500])

In [360]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       10000       0.55      0.64      0.59        56
       10001       0.33      0.25      0.29        36
       10002       0.58      0.45      0.51        31
       10003       0.32      0.52      0.40        87
       10004       0.00      0.00      0.00         2
       10005       0.60      0.62      0.61        52
       10006       0.40      0.39      0.40        44
       10007       0.00      0.00      0.00         2
       10008       0.56      0.70      0.62        46
       10009       0.00      0.00      0.00         4
       10010       0.30      0.27      0.29        11
       10011       0.00      0.00      0.00         7
       10012       0.50      0.25      0.33         4
       10013       0.61      0.46      0.52        37
       10014       1.00      0.17      0.29         6
       10015       0.60      0.62      0.61        24
       10016       0.14      0.07      0.10        14
       10017       0.00    

C:\Users\Doodey\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [361]:
y_pred = model.predict(X_train)
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

       10000       0.84      0.87      0.85       188
       10001       0.81      0.69      0.75       104
       10002       0.72      0.82      0.76        99
       10003       0.69      0.76      0.72       271
       10004       1.00      0.40      0.57        15
       10005       0.72      0.82      0.77       142
       10006       0.80      0.83      0.81       145
       10007       1.00      0.20      0.33         5
       10008       0.80      0.85      0.83       117
       10009       0.75      0.50      0.60        12
       10010       0.58      0.62      0.60        45
       10011       0.00      0.00      0.00         6
       10012       0.71      0.48      0.57        21
       10013       0.81      0.91      0.86        67
       10014       1.00      0.39      0.56        23
       10015       0.85      0.98      0.91        95
       10016       0.79      0.51      0.62        45
       10017       0.89    

C:\Users\Doodey\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [362]:
y_pred = model_new.predict(text_data[1500:])
print(classification_report(Y[1500:], y_pred))

              precision    recall  f1-score   support

       10000       0.62      0.70      0.66        56
       10001       0.40      0.22      0.29        36
       10002       0.65      0.55      0.60        31
       10003       0.30      0.67      0.42        87
       10004       0.00      0.00      0.00         2
       10005       0.65      0.63      0.64        52
       10006       0.54      0.34      0.42        44
       10007       0.00      0.00      0.00         2
       10008       0.64      0.70      0.67        46
       10009       0.00      0.00      0.00         4
       10010       0.75      0.27      0.40        11
       10011       0.00      0.00      0.00         7
       10012       0.50      0.25      0.33         4
       10013       0.71      0.41      0.52        37
       10014       1.00      0.17      0.29         6
       10015       0.68      0.62      0.65        24
       10016       0.50      0.21      0.30        14
       10017       0.00    

In [363]:
y_pred = model_new.predict(text_data[:1500])
print(classification_report(Y[:1500], y_pred))

              precision    recall  f1-score   support

       10000       0.72      0.73      0.73       188
       10001       0.69      0.45      0.55       104
       10002       0.65      0.70      0.67        99
       10003       0.49      0.80      0.61       271
       10004       1.00      0.60      0.75        15
       10005       0.68      0.72      0.70       142
       10006       0.74      0.54      0.63       145
       10007       1.00      0.60      0.75         5
       10008       0.78      0.78      0.78       117
       10009       0.71      0.42      0.53        12
       10010       0.94      0.38      0.54        45
       10011       1.00      0.50      0.67         6
       10012       0.80      0.57      0.67        21
       10013       0.81      0.75      0.78        67
       10014       1.00      0.74      0.85        23
       10015       0.89      0.88      0.89        95
       10016       0.79      0.33      0.47        45
       10017       0.95    